In [68]:
#!/usr/bin/env python
import json, openai,requests
import warnings
import os
warnings.filterwarnings('ignore')
import configparser
import datetime

In [2]:
readKey = configparser.ConfigParser()
readKey.read_file(open('apidata.config'))

openai_org = readKey['OPENAI']["ORG"]
openai_key = readKey['OPENAI']['KEY'] 

In [6]:
def extract_response(org,key,role_player,request_content):
    openai.organization = org
    openai.api_key= key
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", 
             "content": f"You are a {role_player}"},
            {"role": "assistant", 
             "content": f"I have assumed the role of {role_player}"},
            {"role": "user", 
             "content": f"""If you can provide the Knowledge Graph in JSONLD for
                         the dataset I provide, reply 'Yes I can.' """},
            {"role": "assistant", 
             "content": "Yes I can"},
            {"role": "user", 
             "content": f"Here is the dataset {request_content}. Provide Knowledge Graph"}
            ]
        )
    return response['choices'][0]['message']['content']

In [7]:
get_role = input("What role you want me to play: ")

What role you want me to play: Dataset Summarizer


In [8]:
get_role

'Dataset Summarizer'

In [11]:
import tiktoken
import pandas as pd
def num_tokens_from_string(string: str, 
                           encoding_name: str = 'text-davinci-003') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [13]:
#reading in the dataset sample_data.csv

with open('space_titanic.csv','r') as sd:
    data = sd.readlines()

In [14]:
data_chunks = []
rng1 = 1

In [24]:
##Version 1 of the prompt

In [20]:
for x in list(range(1,3)):
    rng2 = rng1 + 5
    print(f'before spliting data {rng1} and {rng2}')
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    print(data_t)
    prompt_made = f"""
    dataset_{x}={data_t}
    
    Prompt:Refer to the [dataset_{x}] and provide the Knowledge graph that contains the 
    details of all the columns in the dataset.    
    Answer:
    """
    data_chunks.append(prompt_made)
    print(f'Number of tokens for the dataset {x} is {num_tokens_from_string(prompt_made)}')
    rng1 = rng1 + 5

before spliting data 21 and 26
PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0017_02,Earth,False,F/6/P,55 Cancri e,14.0,False,412.0,0.0,1.0,0.0,679.0,Philda Brighttt,False\n', '0020_01,Earth,True,E/0/S,TRAPPIST-1e,1.0,False,0.0,0.0,0.0,0.0,0.0,Almary Brantuarez,False\n', '0020_02,Earth,True,E/0/S,55 Cancri e,49.0,False,0.0,0.0,0.0,0.0,0.0,Glendy Brantuarez,False\n', '0020_03,Earth,True,E/0/S,55 Cancri e,29.0,False,0.0,0.0,,0.0,0.0,Mollen Mcfaddennon,False\n', '0020_04,Earth,False,E/0/S,TRAPPIST-1e,10.0,False,0.0,0.0,0.0,0.0,0.0,Breney Jacostanley,True\n']
Number of tokens for the dataset 1 is 388
before spliting data 26 and 31
PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0020_05,Earth,True,E/0/S,PSO J318.5-22,1.0,False,,0.0,0.0,0.0,0.0,Mael Brantuarez,False\n', '0020_06,Earth,False,E/0/S,TRAPPIST-1e,7.0,False,0.0,0.0,0.0,0.0,0.0,

In [16]:
def chunk_to_df(chunk_num,data_chunk):
    
    start = data_chunk.split('\\n')
    
    clean_step1 = start[0].strip(' \n').replace('=','').replace(f'dataset_{chunk_num}','')

    clean_step2 = clean_step1.split('\n+[')
    
    header = clean_step2[0]
    row1 = clean_step2[1]
    
    clean_row = []
    for dat in start[1:]:
        clean_row.append(dat.strip(" ',"))
        
    clean_row.insert(0,row1.strip("'"))
    clean_row.insert(0,header)
    
    clean_data_list = [data.split(',') for data in clean_row]
    
    chunk_df = pd.DataFrame(clean_data_list[1:-1],columns=clean_data_list[0])
    
    return chunk_df

In [18]:
chunk_to_df(1,data_chunks[0])

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [22]:
content_output = extract_response(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=data_chunks[0])

In [23]:
content_output

'Here is the JSON-LD Knowledge Graph for the dataset you provided:\n\n```\n{\n  "@context": {\n    "schema": "http://schema.org/",\n    "passenger": "schema:Person",\n    "identifier": "schema:identifier",\n    "homePlanet": "schema:homeLocation",\n    "cryoSleep": "schema:isRelatedTo",\n    "cabin": "schema:occupationalCategory",\n    "destination": "schema:destination",\n    "age": "schema:age",\n    "vip": "schema:isVIP",\n    "roomService": "schema:hasRoomService",\n    "foodCourt": "schema:hasFoodCourt",\n    "shoppingMall": "schema:hasShoppingMall",\n    "spa": "schema:hasSpa",\n    "vrDeck": "schema:hasVRDeck",\n    "name": "schema:name",\n    "transported": "schema:isTransported"\n  },\n  "@type": "passenger",\n  "identifier": "PassengerId",\n  "homePlanet": "HomePlanet",\n  "cryoSleep": "CryoSleep",\n  "cabin": "Cabin",\n  "destination": "Destination",\n  "age": "Age",\n  "vip": "VIP",\n  "roomService": "RoomService",\n  "foodCourt": "FoodCourt",\n  "shoppingMall": "ShoppingMa

In [ ]:
## Version 2 of the prompt

In [28]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset_{x}={data_t}

Prompt:Refer to the [dataset] and summarize it as a Knowledge graph that contains the 
details of all the columns in the dataset, and your inference of the dataset    
Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 31 and 36
Number of tokens for the dataset is 397

dataset_2=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0026_01,Europa,False,C/0/P,55 Cancri e,34.0,False,22.0,6073.0,0.0,1438.0,328.0,Anyoni Unconary,False\n', '0028_01,Mars,False,F/8/P,TRAPPIST-1e,43.0,False,1125.0,0.0,136.0,48.0,0.0,Ceros Mare,False\n', '0030_01,Earth,False,G/4/S,TRAPPIST-1e,32.0,False,0.0,850.0,81.0,437.0,453.0,Ginia Morsentley,False\n', '0031_01,Mars,False,F/9/P,TRAPPIST-1e,47.0,False,214.0,0.0,1411.0,0.0,1229.0,Coobix Datie,True\n', '0031_02,Mars,False,F/9/P,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,0.0,0.0,Cinets Datie,True\n']

Prompt:Refer to the [dataset] and summarize it as a Knowledge graph that contains the 
details of all the columns in the dataset, and your inference of the dataset    
Answer:



In [30]:
content_output_2 = extract_response(org=openai_org,key=openai_key,
                                role_player=get_role, 
                                  request_content=prompt_made)

In [31]:
content_output_2

'Here\'s the Knowledge Graph in JSON-LD format for the given dataset:\n\n```\n{\n  "@context": {\n    "schema": "http://schema.org/"\n  },\n  "@type": "schema:Dataset",\n  "schema:name": "Passenger Information",\n  "schema:description": "A dataset containing information about passengers travelling to TRAPPIST-1e from various planets.",\n  "schema:creator": "Your Name",\n  "schema:variableMeasured": {\n    "@type": "schema:Property",\n    "schema:name": "Passenger Attributes",\n    "schema:description": "The attributes of each passenger in the dataset.",\n    "schema:variableMeasured": ["HomePlanet", "CryoSleep", "Cabin", "Destination", "Age", "VIP", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck", "Name", "Transported"]\n  },\n  "schema:includedInDataCatalog": {\n    "@type": "schema:DataCatalog",\n    "schema:name": "Some Dataset Repository",\n    "schema:url": "http://example.com"\n  }\n}\n```\n\nFrom the given dataset, we can infer that the dataset contains information a

In [32]:
#version 3 of the prompt

In [34]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and list 50 statistical, factual and inferential questions    
Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 31 and 36
Number of tokens for the dataset is 378

dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0026_01,Europa,False,C/0/P,55 Cancri e,34.0,False,22.0,6073.0,0.0,1438.0,328.0,Anyoni Unconary,False\n', '0028_01,Mars,False,F/8/P,TRAPPIST-1e,43.0,False,1125.0,0.0,136.0,48.0,0.0,Ceros Mare,False\n', '0030_01,Earth,False,G/4/S,TRAPPIST-1e,32.0,False,0.0,850.0,81.0,437.0,453.0,Ginia Morsentley,False\n', '0031_01,Mars,False,F/9/P,TRAPPIST-1e,47.0,False,214.0,0.0,1411.0,0.0,1229.0,Coobix Datie,True\n', '0031_02,Mars,False,F/9/P,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,0.0,0.0,Cinets Datie,True\n']

Prompt:Refer to the [dataset] and list 50 statistical, factual and inferential questions    
Answer:



In [35]:
inference_3 = extract_response(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [37]:
questions_data = inference_3.split('\n\n')[1]

In [38]:
questions_data

'1. How many passengers are there in the dataset?\n2. What is the average age of the passengers?\n3. Which planet has the most number of passengers in the dataset?\n4. What is the most common Cabin type for the passengers?\n5. What is the most popular Destination among the passengers?\n6. What is the average shopping mall expense of the passengers?\n7. How many passengers used VIP services?\n8. Which passenger traveled the longest distance?\n9. What is the total amount of food court expenses incurred by all passengers?\n10. How many passengers used the Spa facility?\n11. How many passengers are in CryoSleep?\n12. What is the highest amount spent on Food Court by a passenger?\n13. What is the most common Transported status among the passengers?\n14. What is the average age of passengers who used Room Service?\n15. What is the average age of passengers who used VRDeck facility?\n16. How many passengers used the Transported service?\n17. How many passengers are on Mars?\n18. What is the a

In [39]:
###Follow up prompt

In [47]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and answer the following questions. 
{questions_data}
At the end print COMPLETED INFERENCE

Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 31 and 36
Number of tokens for the dataset is 1120

dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0026_01,Europa,False,C/0/P,55 Cancri e,34.0,False,22.0,6073.0,0.0,1438.0,328.0,Anyoni Unconary,False\n', '0028_01,Mars,False,F/8/P,TRAPPIST-1e,43.0,False,1125.0,0.0,136.0,48.0,0.0,Ceros Mare,False\n', '0030_01,Earth,False,G/4/S,TRAPPIST-1e,32.0,False,0.0,850.0,81.0,437.0,453.0,Ginia Morsentley,False\n', '0031_01,Mars,False,F/9/P,TRAPPIST-1e,47.0,False,214.0,0.0,1411.0,0.0,1229.0,Coobix Datie,True\n', '0031_02,Mars,False,F/9/P,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,0.0,0.0,Cinets Datie,True\n']

Prompt:Refer to the [dataset] and answer the following questions. 
1. How many passengers are there in the dataset?
2. What is the average age of the passengers?
3. Which planet has the most number of passengers in the dataset?
4. What is the most common Cabin type for the passengers?
5. What is th

In [44]:
def extract_summary(org,key,role_player,request_content):
    openai.organization = org
    openai.api_key= key
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", 
             "content": f"You are a {role_player}"},
            {"role": "assistant", 
             "content": f"I have assumed the role of {role_player}"},
            {"role": "user", 
             "content": f"Here is the dataset {request_content}."}
            ]
        )
    return response['choices'][0]['message']['content']

In [48]:
designed_summary =  extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [49]:
designed_summary

'1. There are 5 passengers in the dataset.\n2. The average age of the passengers is 31.6.\n3. TRAPPIST-1e has the most number of passengers in the dataset.\n4. F/9/P is the most common Cabin type for the passengers.\n5. TRAPPIST-1e is the most popular Destination among the passengers.\n6. The average shopping mall expense of the passengers is 1,129.4.\n7. None of the passengers used VIP services.\n8. Anyoni Unconary traveled the longest distance.\n9. The total amount of food court expenses incurred by all passengers is 7,508.\n10. Two passengers used the Spa facility.\n11. None of the passengers are in CryoSleep.\n12. 1411.0 is the highest amount spent on Food Court by a passenger.\n13. False is the most common Transported status among the passengers.\n14. The average age of passengers who used Room Service is 11.0.\n15. The average age of passengers who used VRDeck facility is 328.0.\n16. None of the passengers used the Transported service.\n17. Two passengers are on Mars.\n18. There 

In [ ]:
#check if gpt can make such inferences directly

In [50]:
rng2 = rng1 + 5
print(f'before spliting data {rng1} and {rng2}')
data_t = f'{data[0]}+{data[rng1:rng2]}'
#print(data_t)
prompt_made = f"""
dataset={data_t}

Prompt:Refer to the [dataset] and list 50 statistical, factual and inferences    
Answer:
"""
print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
print(prompt_made)

before spliting data 31 and 36
Number of tokens for the dataset is 377

dataset=PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
+['0026_01,Europa,False,C/0/P,55 Cancri e,34.0,False,22.0,6073.0,0.0,1438.0,328.0,Anyoni Unconary,False\n', '0028_01,Mars,False,F/8/P,TRAPPIST-1e,43.0,False,1125.0,0.0,136.0,48.0,0.0,Ceros Mare,False\n', '0030_01,Earth,False,G/4/S,TRAPPIST-1e,32.0,False,0.0,850.0,81.0,437.0,453.0,Ginia Morsentley,False\n', '0031_01,Mars,False,F/9/P,TRAPPIST-1e,47.0,False,214.0,0.0,1411.0,0.0,1229.0,Coobix Datie,True\n', '0031_02,Mars,False,F/9/P,TRAPPIST-1e,2.0,False,0.0,0.0,0.0,0.0,0.0,Cinets Datie,True\n']

Prompt:Refer to the [dataset] and list 50 statistical, factual and inferences    
Answer:



In [51]:
inference_possiblity =  extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=prompt_made)

In [53]:
inference_possiblity

'1. The dataset contains information on passengers, including their PassengerId, HomePlanet, CryoSleep, Cabin, Destination, Age, VIP status, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck, Name, and whether they were Transported.\n2. There are at least five passengers in the dataset, with PassengerIds 0026_01, 0028_01, 0030_01, 0031_01, and 0031_02.\n3. The HomePlanet for each passenger is either Europa, Mars, or Earth.\n4. None of the passengers in the dataset were in CryoSleep.\n5. The Cabin for each passenger is either C/0/P, F/8/P, G/4/S, or F/9/P.\n6. The Destination for each passenger is TRAPPIST-1e, a potentially habitable exoplanet located about 39 light years away from Earth.\n7. The Age for each passenger ranges from 2 to 47 years.\n8. None of the passengers in the dataset are VIPs.\n9. Some passengers in the dataset received RoomService, FoodCourt, ShoppingMall, Spa, or VRDeck amenities during their travels.\n10. At least one passenger in the dataset had the name Anyoni U

In [54]:
#It infers in its own method. But the earlier 50 question guided it

In [56]:
inference_requests = []
rng1 = 1
for x in list(range(1,5)):
    rng2 = rng1 + 30
    print(f'before spliting data {rng1} and {rng2}')
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    #print(data_t)
    prompt_made = f"""
    dataset={data_t}

    Prompt:Refer to the [dataset] and answer the following questions. 
    {questions_data}
    At the end print COMPLETED INFERENCE

    Answer:
    """
    print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
    inference_requests.append(prompt_made)
    rng1 = rng1 + 30

before spliting data 1 and 31
Number of tokens for the dataset is 2574
before spliting data 31 and 61
Number of tokens for the dataset is 2571
before spliting data 61 and 91
Number of tokens for the dataset is 2586
before spliting data 91 and 121
Number of tokens for the dataset is 2566


In [58]:
prompt_questions = []
rng1 = 1
for x in list(range(1,5)):
    rng2 = rng1 + 30
    data_t = f'{data[0]}+{data[rng1:rng2]}'
    prompt_made = f"""
    dataset={data_t}

    Prompt:Refer to the [dataset] and make the phy-beta-gamma version of inferences.

    Answer:
    """
    print(f'Number of tokens for the dataset is {num_tokens_from_string(prompt_made)}')
    prompt_questions.append(prompt_made)
    rng1 = rng1 + 30

Number of tokens for the dataset is 1835
Number of tokens for the dataset is 1832
Number of tokens for the dataset is 1847
Number of tokens for the dataset is 1827


In [57]:
len(inference_requests)

4

In [61]:
for ind,inf in enumerate(inference_requests):
    comp = extract_summary(org=openai_org, key=openai_key, 
                 role_player=get_role, request_content=inf)
    prompt_dict = {'prompt':prompt_questions[ind],
                  'completion':comp}
    with open('inferences_prompt.jsonl','a+') as inje:
        json.dump(prompt_dict,fp=inje)
        inje.write('\n')

In [64]:
def file_upload(filename, purpose='fine-tune'):
    resp = openai.File.create(purpose=purpose,file=open(file=filename,
                                                        mode='rb'))
    print(resp)
    return resp

def finetune_model(fileId, suffix,model='davinci',api_key=openai_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    payload = {'training_file':fileId,'model':model,'suffix':suffix}
    resp = requests.request(method='POST',
                            url='https://api.openai.com/v1/fine-tunes',
                           headers=header,
                           json=payload,timeout=40)
    print(resp.json())
    
def finetune_get(ftId,api_key=openai_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}',
                           headers=header,timeout=40)
    print(resp.json())
    
def finetune_event(ftId,api_key=openai_key):
    header = {'Content-Type':'application/json',
              'Authorization':f'Bearer {api_key}'}
    resp = requests.request(method='GET',
                url=f'https://api.openai.com/v1/fine-tunes/{ftId}/events',
                           headers=header,timeout=40)
    print(resp.json())

In [65]:
file_upload(filename='inferences_prompt.jsonl',)

{
  "bytes": 26165,
  "created_at": 1678466823,
  "filename": "file",
  "id": "file-7eEqgLRz0Mki8F1xFzd2oz48",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


<File file id=file-7eEqgLRz0Mki8F1xFzd2oz48 at 0x7fa8cc96e390> JSON: {
  "bytes": 26165,
  "created_at": 1678466823,
  "filename": "file",
  "id": "file-7eEqgLRz0Mki8F1xFzd2oz48",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}

In [69]:
finetune_model(fileId='file-7eEqgLRz0Mki8F1xFzd2oz48',
               suffix='phy-beta-gamma')

{'object': 'fine-tune', 'id': 'ft-pUzhDaGmfikioguqgcRVjYLx', 'hyperparams': {'n_epochs': 4, 'batch_size': None, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': None}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-7eEqgLRz0Mki8F1xFzd2oz48', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 26165, 'created_at': 1678466823, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [], 'created_at': 1678466919, 'updated_at': 1678466919, 'status': 'pending', 'fine_tuned_model': None, 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-pUzhDaGmfikioguqgcRVjYLx', 'created_at': 1678466919}]}


In [71]:
finetune_get(ftId='ft-pUzhDaGmfikioguqgcRVjYLx')

{'object': 'fine-tune', 'id': 'ft-pUzhDaGmfikioguqgcRVjYLx', 'hyperparams': {'n_epochs': 4, 'batch_size': None, 'prompt_loss_weight': 0.01, 'learning_rate_multiplier': None}, 'organization_id': 'org-QWWIpNe4Z3KsRVI0YR6lhyJW', 'model': 'davinci', 'training_files': [{'object': 'file', 'id': 'file-7eEqgLRz0Mki8F1xFzd2oz48', 'purpose': 'fine-tune', 'filename': 'file', 'bytes': 26165, 'created_at': 1678466823, 'status': 'processed', 'status_details': None}], 'validation_files': [], 'result_files': [], 'created_at': 1678466919, 'updated_at': 1678466919, 'status': 'pending', 'fine_tuned_model': None, 'events': [{'object': 'fine-tune-event', 'level': 'info', 'message': 'Created fine-tune: ft-pUzhDaGmfikioguqgcRVjYLx', 'created_at': 1678466919}]}
